In [1]:
# pip install pandas
import pandas as pd
import numpy as np

# pip install -U scikit-learn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

import yaml

import functions    # local functions in this repository

# import new transactions
Import new transactions into the database, either from csv or via API (comdirect bank).  
The new transactions are categorized based on the description text using Machine Learning, trained with the existing transactions.  
The account balance is also updated.

## read settings from config file

In [2]:
with open("config.ini", "r") as ymlfile:
    cfg = yaml.safe_load(ymlfile)

clm = cfg['column names database']

## pre-processing data

import new transactions, either:
- from csv (adjust as needed. start from the top and uncomment line by line)

In [3]:
transactions_new = pd.read_csv( cfg['CSV filenames']['bank export'] + '.csv', encoding = 'ISO-8859-1', sep =';', decimal=',', thousands='.', header=3 )

# transactions_new = transactions_new.dropna(how='all', axis=1)   # removes all empty columns

# transactions_new = transactions_new.drop(['Buchungstag' , ], axis=1)   # insert list of column-names to remove

# transactions_new = transactions_new.rename({
#     'Wertstellung (Valuta)' : clm['date']   ,
#     'Umsatz in EUR'         : clm['amount'] ,
#     'Vorgang'               : clm['type']   ,
#     'Buchungstext'          : clm['text']   ,
#     } , axis=1)

# transactions_new[clm['date']] = pd.to_datetime( transactions_new[clm['date']] , format='%d.%m.%Y', errors='coerce' )

# transactions_new = transactions_new.dropna(how='any', axis=0)   # removes rows with empty entries


transactions_new    # prints out imported transactions in Jupyter notebook

- comdirect API

In [4]:
# transactions_new = functions.transactions_API_comdirect(clm, pastDays = 30)

import csv database

In [5]:
transactions = pd.read_csv( cfg['CSV filenames']['database']  + '.csv', encoding = 'ISO-8859-1' )
transactions[clm['date']] = pd.to_datetime( transactions[clm['date']], format = cfg['date format'] )

# get categories
categories = pd.DataFrame( transactions[clm['category']].unique() ).sort_values(0)
categories.to_csv( cfg['CSV filenames']['categories'] + '.csv', encoding = "ISO-8859-1", index=0, header=0 )

prepare merge

In [6]:
# filtering time
max_date = max(transactions[clm["date"]])
transactions_new = transactions_new[transactions_new[clm['date']] >= max_date]

# removing overlap
transactions_new = transactions.merge(transactions_new, on=[clm['type'], clm['date'], clm['text'], clm['amount']], how='right', indicator=True )
transactions_new = transactions_new.query('_merge == "right_only"').drop(['_merge'], axis=1)

# sort to prevent negative balance due to same day transactions
transactions_new.sort_values([clm['date'], clm['amount']], ascending = [False, True], inplace = True)

## train category classifier

In [7]:
# text pre-processing
keywords_train = functions.PreProcText(transactions[clm['text']] , minwordlength=3)

# feature extraction
vectorizer = CountVectorizer(ngram_range=(1,1), max_features = 500)
y_train = transactions[clm['category']]
X_train = vectorizer.fit_transform(keywords_train).toarray()
X_train = np.column_stack(( X_train , transactions[clm['amount']].to_list() ))    # add amount

# model training
classifier = RandomForestClassifier(100)
classifier.fit(X_train, y_train)

RandomForestClassifier()

## categorize new transactions

In [8]:
# text pre-processing
keywords_new = functions.PreProcText(transactions_new[clm['text']] , minwordlength=3)

# feature extraction
X_new = vectorizer.transform(keywords_new).toarray()
X_new = np.column_stack(( X_new , transactions_new[clm['amount']].to_list() ))

# classification
transactions_new[clm['category']] = classifier.predict(X_new)

## output

In [9]:
# calculate balance
final_balance = transactions.iloc[0][clm['balance']]
transactions_new[clm['balance']] = transactions_new.loc[::-1, clm['amount']].cumsum()[::-1] + final_balance

transactions_new[clm['new']] = True

transactions_new

In [10]:
transactions.head(10)

## merge & save

In [11]:
transactions = pd.concat( [transactions_new, transactions] ).reset_index(drop=True)

In [12]:
transactions.to_csv(cfg['CSV filenames']['database'] + '.csv', encoding = "ISO-8859-1", index=0)